Note: To display the density grid calculations, in VisIt, load in the .cube file (must select file type and deselect default option), and deselect options to apply options to all plots/windows. When making density grid plots, use the minimum and maximum energy values displayed inthe legend (don't turn off annotations in VisIt immediately) to *normalize* the color gradient. Make a low quality plot and find a good viewing angle; then, increase the resolution and print screen as png. 

TODO: try making density grid plots using alternative software posted in slack.

In [1]:
using PorousMaterials
using JLD2

In [2]:
@eval PorousMaterials PATH_TO_CRYSTALS=joinpath(pwd(),"structural_relaxation","post-relaxation_cifs")
# @eval PorousMaterials PATH_TO_CRYSTALS=joinpath(pwd(), "images/grid_images/replicated_xtals/")

"/home/ng/DTRA/structural_relaxation/post-relaxation_cifs"

In [3]:
###
#  Set flag for desired calculation
###
calc_energy_grid = true   # flag for energy grid calculations
write_density_grid = false # flag for extracting density grid from *.jld2 sim files 

false

In [4]:
# preset dictionary vlues
params = Dict(:adsorbate   => "Xe",
              :ljff        => "UFF",
              :grid_points => (100, 100, 100), 
              :repfactors  => (1, 2, 2),
              :temperature => 298.0, 
              :n_rotations => 750,
              :energy_min  => Tuple{Float64,CartesianIndex{3}},
              :xyz_file    => true,
              :vtk_file    => true)

Dict{Symbol,Any} with 9 entries:
  :ljff        => "UFF"
  :adsorbate   => "Xe"
  :grid_points => (100, 100, 100)
  :temperature => 298.0
  :energy_min  => Tuple{Float64,CartesianIndex{3}}
  :xyz_file    => true
  :vtk_file    => true
  :repfactors  => (1, 2, 2)
  :n_rotations => 750

In [5]:
# I need to put place the adsorbate at the location of the minimum energy
function adsorbate_in_mof(energy_grid_params::Dict{String,Dict{Symbol,Any}}, xtal::Crystal, grid::Grid)
    filename = energy_grid_params[xtal.name][:adsorbate] * "_in_" * xtal.name
    
    molecule = Molecule(energy_grid_params[xtal.name][:adsorbate])
    xyz = Tuple([energy_grid_params[xtal.name][:energy_min][2][i] for i in 1:3])
    
    xf_minE = id_to_xf(xyz, grid.n_pts) # fractional coords of min
    x_minE  = Cart(Frac(xf_minE), xtal.box) # Cartesian coords of min
    # make struct
    xe_atom_c = Atoms(1, [molecule.species], x_minE) 

    return write_xyz(xe_atom_c, filename)
end

adsorbate_in_mof (generic function with 1 method)

In [6]:
# min_E, argmin_E = findmin(grid.data)
# xyz = Tuple([argmin_E[i] for i in 1:3])
# xf_minE = id_to_xf(xyz, grid.n_pts)
# x_minE  = Cart(Frac(xf_minE), crystal.box) 
# xe_atom_c = Atoms(1, [:Xe], x_minE) 
# write_xyz(xe_atom_c, "blah.xyz")

In [7]:
# xtal_names = ["NiPyC2_experiment_sc222.cif", 
#               "NiPyC2_relax_sc222_meta_functionalized_NH2_pbesol_relax.cif",
#               "NiPyC2_relax_sc222_meta_functionalized_CH3_pbesol_relax.cif",
#               "NiPyC2_relax_sc222_meta_functionalized_N-C-O_pbesol_relax.cif"]

xtal_names = ["NiPyC2_relax_sc211_meta_functionalized_C-C_pbesol_relax.cif",
              "NiPyC2_relax_sc211_ortho_functionalized_C-C_pbesol_relax.cif"]

energy_grid_params = Dict(zip(xtal_names, [params for i in 1:length(xtal_names)]))

Dict{String,Dict{Symbol,Any}} with 2 entries:
  "NiPyC2_relax_sc211_orth… => Dict{Symbol,Any}(:ljff=>"UFF",:adsorbate=>"Xe",:…
  "NiPyC2_relax_sc211_meta… => Dict{Symbol,Any}(:ljff=>"UFF",:adsorbate=>"Xe",:…

## Density Grid from `.jld2` File

In [8]:
if write_density_grid
    for xtal in xtal_names
        # load crystal and replicate 
        crx = Crystal(xtal)
    #     energy_grid_params[xtal][:repfactors] = replication_factors(crystal, 14.0)
        crystal = replicate(crx, energy_grid_params[xtal][:repfactors])
        strip_numbers_from_atom_labels!(crystal)

        # assign energy calculation parameters
        mol    = Molecule(energy_grid_params[xtal][:adsorbate])
        ljff   = LJForceField(energy_grid_params[xtal][:ljff])
        points = energy_grid_params[xtal][:grid_points]
        temp   = energy_grid_params[xtal][:temperature]
        rot    = energy_grid_params[xtal][:n_rotations]

    #     pmin      = -2 # minimum pessure in log10, units: bar
    #     pmax      = 1.1 # maximum pressure (actual value), units: bar
    #     nstep     = 15 # number of intervals in pressure range
    #     pressures = 10 .^ range(pmin, stop=log10(pmax), length=nstep)

        pressures = [0.01, 0.1, 1.1]
        nburn     = 25000
        nsample   = 25000

        # loop over pressures
        for p in 1:length(pressures)
            filename = μVT_output_filename(crystal, mol, temp, pressures[p], ljff, nburn, nsample)
            println(filename)

            @load joinpath(PorousMaterials.PATH_TO_SIMS, filename) results

            grid = results["density grid"]

            # find the value and location of the minimum energy 
            energy_grid_params[xtal][:energy_min] = findmin(grid.data)
            println(xtal, " energy min = ", energy_grid_params[xtal][:energy_min])

            # write cube file using crystal name for filename
            write_cube(grid, filename)
        end

        # write xyz file
        if energy_grid_params[xtal][:xyz_file]
            atoms_c = Cart(crystal.atoms, crystal.box)
            write_xyz(crystal)
        end

        # write vtk file
        if energy_grid_params[xtal][:vtk_file]
            write_vtk(crystal)
        end

    #     adsorbate_in_mof(energy_grid_params, crystal, grid)
    end
end

## Energy Grid Calculations

In [9]:
if calc_energy_grid
    for xtal in xtal_names
        # load crystal and replicat (if needed)
        crystal = replicate(Crystal(xtal), energy_grid_params[xtal][:repfactors])
        strip_numbers_from_atom_labels!(crystal)

        # assign energy calculation parameters
        mol    = Molecule(energy_grid_params[xtal][:adsorbate])
        ljff   = LJForceField(energy_grid_params[xtal][:ljff])
        points = energy_grid_params[xtal][:grid_points]
        temp   = energy_grid_params[xtal][:temperature]
        rot    = energy_grid_params[xtal][:n_rotations]

        # perform energy grid calculation
        grid = energy_grid(crystal, mol, ljff; n_pts=points, temperature=temp, n_rotations=rot)

        # find the value and location of the minimum energy 
        energy_grid_params[xtal][:energy_min] = findmin(grid.data)
        println(xtal, " energy min = ", energy_grid_params[xtal][:energy_min])

        # write cube file using crystal name for filename
        write_cube(grid, "Egrid_" * crystal.name)

        # write xyz file
        if energy_grid_params[xtal][:xyz_file]
            atoms_c = Cart(crystal.atoms, crystal.box)
            write_xyz(crystal)
        end

        # write vtk file
        if energy_grid_params[xtal][:vtk_file]
            write_vtk(crystal)
        end

    #     adsorbate_in_mof(energy_grid_params, crystal, grid)
    end
end

Computing energy grid of Xe in NiPyC2_relax_sc211_meta_functionalized_C-C_pbesol_relax.cif
	Regular grid (in fractional space) of 100 by 100 by 100 points superimposed over the unit cell.
NiPyC2_relax_sc211_meta_functionalized_C-C_pbesol_relax.cif energy min = (-35.83464644235688, CartesianIndex(73, 14, 92))
	See /home/ng/DTRA/data/grids/Egrid_NiPyC2_relax_sc211_meta_functionalized_C-C_pbesol_relax.cif.cube
See NiPyC2_relax_sc211_meta_functionalized_C-C_pbesol_relax.vtk
Computing energy grid of Xe in NiPyC2_relax_sc211_ortho_functionalized_C-C_pbesol_relax.cif
	Regular grid (in fractional space) of 100 by 100 by 100 points superimposed over the unit cell.
NiPyC2_relax_sc211_ortho_functionalized_C-C_pbesol_relax.cif energy min = (-30.50230912465404, CartesianIndex(30, 15, 36))
	See /home/ng/DTRA/data/grids/Egrid_NiPyC2_relax_sc211_ortho_functionalized_C-C_pbesol_relax.cif.cube
See NiPyC2_relax_sc211_ortho_functionalized_C-C_pbesol_relax.vtk
